In [ ]:
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt
import timeit

<h1>Load training, validation, testing set from your preprocessed files</h1>

In [ ]:
x_train, y_train = pickle.load(open('cat_dog_train.p','rb'))

x_valid, y_valid = pickle.load(open('cat_dog_valid.p','rb'))
x_test = pickle.load(open('cat_dog_test.p','rb'))

<h1>Define hyperparameter</h1>

In [ ]:
learning_rate = 0.001
batch_size = 64

image_width = 227
image_height = 227
image_depth = 3
num_labels = 2

<h1>Placeholder</h1>

In [ ]:
tf.reset_default_graph()

network_input = tf.placeholder(tf.float32, shape=[None, 227, 227, 3])
true_labels = tf.placeholder(tf.float32, shape=[None, 2])
keep_prob = tf.placeholder(tf.float32)

In [ ]:
def get_alexnet_variables(output_class_num):
    with tf.name_scope('conv1'):
        w1 = tf.Variable(tf.truncated_normal(shape=[11, 11, 3, 96], stddev=0.1), name='w1')
        b1 = tf.Variable(tf.zeros([96]), name='b1')
    with tf.name_scope('conv2'):
        w2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 96, 256], stddev=0.1), name='w2')
        b2 = tf.Variable(tf.constant(1.0, shape=[256]), name='b2')
    with tf.name_scope('conv3'):
        w3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 256, 384], stddev=0.1), name='w3')
        b3 = tf.Variable(tf.zeros([384]), name='b3')
    with tf.name_scope('conv4'):
        w4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 384, 384], stddev=0.1), name='w4')
        b4 = tf.Variable(tf.constant(1.0, shape=[384]), name='b4')
    with tf.name_scope('conv5'):
        w5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 384, 256], stddev=0.1), name='w5')
        b5 = tf.Variable(tf.zeros([256]), name='b5')
    with tf.name_scope('fc6'):
        w6 = tf.Variable(tf.truncated_normal(shape=[(224 // 2 ** 5) * (224 // 2 ** 5) * 256, 4096], stddev=0.1),
                        name='w6')
        b6 = tf.Variable(tf.constant(1.0, shape=[4096]), name='b6')
    with tf.name_scope('fc7'):
        w7 = tf.Variable(tf.truncated_normal(shape=[4096, 4096], stddev=0.1), name='w7')
        b7 = tf.Variable(tf.constant(1.0, shape=[4096]), name='b7')
    with tf.name_scope('fc8'):
        w8 = tf.Variable(tf.truncated_normal(shape=[4096, output_class_num], stddev=0.1), name='w8')
        b8 = tf.Variable(tf.constant(1.0, shape=[output_class_num]), name='b8')

    variables = dict(w1=w1, w2=w2, w3=w3, w4=w4, w5=w5, w6=w6, w7=w7, w8=w8, b1=b1, b2=b2, b3=b3, b4=b4, b5=b5, b6=b6,
                    b7=b7, b8=b8)

    return variables


<h1>AlexNet</h1>

In [ ]:
def alexnet(variables, input_data, keep_prob_for_net):
    # layer one : conventional layer
    with tf.name_scope('conv1'):
        conv1 = tf.nn.conv2d(input_data, variables['w1'], strides=[1, 4, 4, 1], padding='SAME')
        conv1 = tf.nn.bias_add(conv1, variables['b1'])
        conv1 = tf.nn.relu(conv1)
        conv1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv1 = tf.nn.local_response_normalization(conv1)

    # layer two : conventional layer
    with tf.name_scope('conv2'):
        conv2 = tf.nn.conv2d(conv1, variables['w2'], strides=[1, 1, 1, 1], padding='SAME')
        conv2 = tf.nn.bias_add(conv2, variables['b2'])
        conv2 = tf.nn.relu(conv2)
        conv2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv2 = tf.nn.local_response_normalization(conv2)

    # layer three: conventional layer
    with tf.name_scope('conv3'):
        conv3 = tf.nn.conv2d(conv2, variables['w3'], strides=[1, 1, 1, 1], padding='SAME')
        conv3 = tf.nn.bias_add(conv3, variables['b3'])
        conv3 = tf.nn.relu(conv3)

    # layer four: conventional layer
    with tf.name_scope('conv4'):
        conv4 = tf.nn.conv2d(conv3, variables['w4'], strides=[1, 1, 1, 1], padding='SAME')
        conv4 = tf.nn.bias_add(conv4, variables['b4'])
        conv4 = tf.nn.relu(conv4)

    # layer five: conventional layer
    with tf.name_scope('conv5'):
        conv5 = tf.nn.conv2d(conv4, variables['w5'], strides=[1, 1, 1, 1], padding='SAME')
        conv5 = tf.nn.bias_add(conv5, variables['b5'])
        conv5 = tf.nn.relu(conv5)
        conv5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv5 = tf.nn.local_response_normalization(conv5)

    # flat data
    with tf.name_scope('flatten'):
        flatten = tf.reshape(conv5, shape=[-1, (224 // 2 ** 5) * (224 // 2 ** 5) * 256])

    # layer six: fully connected layer
    with tf.name_scope('fc6'):
        fc6 = tf.matmul(flatten, variables['w6']) + variables['b6']
        fc6 = tf.nn.tanh(fc6)
        fc6 = tf.nn.dropout(fc6, keep_prob=keep_prob_for_net)

    # layer seven: fully connected layer
    with tf.name_scope('fc7'):
        fc7 = tf.matmul(fc6, variables['w7']) + variables['b7']
        fc7 = tf.nn.tanh(fc7)
        fc7 = tf.nn.dropout(fc7, keep_prob=keep_prob_for_net)

    # layer eight: fully connected layer
    with tf.name_scope('fc8'):
        logits_output = tf.matmul(fc7, variables['w8']) + variables['b8']

    return logits_output

<h1>Cost and Optimization</h1>

<h1>Training and validation</h1>
<h2>Train your model only 10 epochs</h2>
<p style="font-size:20px">1. Print out training accuracy and validation accuracy each training epoch</p>
<p style="font-size:20px">2. Print out training time each training epoch</p>
<p style="font-size:20px">3. Your goal is to reach 85% validation accuracy in 10 training epochs. If you reach that, you can perform testing, print out your test accuracy. Plot out the ten images with title that contains the probability of the labeled class.</p>